## Setting up the environment and installing necessary libraries

In [1]:
!git clone https://github.com/CarperAI/trlx.git
!git config --global --add safe.directory /content/trlx && cd /content/trlx && pip install -e .

# uninstall scikit_learn + jax to avoid numpy issues
!pip uninstall -y scikit_learn jax

import os

# run within repo
os.chdir('/content/trlx/examples/summarize_rlhf/')
print(os.getcwd())

!pip install -r requirements.txt
!pip install mpi4py

# run within reward model directory
os.chdir('/content/trlx/examples/summarize_rlhf/reward_model/')
print(os.getcwd())

Cloning into 'trlx'...
remote: Enumerating objects: 8089, done.
remote: Counting objects: 100% (3366/3366), done.
remote: Compressing objects: 100% (764/764), done.
remote: Total 8089 (delta 2857), reused 2896 (delta 2600), pack-reused 4723
Receiving objects: 100% (8089/8089), 46.84 MiB | 21.32 MiB/s, done.
Resolving deltas: 100% (5563/5563), done.
Obtaining file:///content/trlx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.3/867.3 kB 63.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

## Creating a custom dataset

In [2]:
from transformers import pipeline, set_seed
import json

def generate_examples(prompt_list, model_name='gpt2', max_length=50, num_return_sequences=2, seed=42):
    generator = pipeline('text-generation', model=model_name, device=0)
    set_seed(seed)
    examples = []
    for prompt in prompt_list:
        result = generator(prompt, max_length=max_length, num_return_sequences=num_return_sequences)
        example = {'prompt': prompt}
        for i, res in enumerate(result):
            answer = res['generated_text'].lstrip().removeprefix(prompt).strip()
            example[f'answer{i + 1}'] = answer
        examples.append(example)
        print(json.dumps(example, indent=2))
    return examples

In [3]:
prompts = [
    "Among which hypoxia AV 0 difference is max ?",
    "Which of the following needs cholesterol and other lipids for growth?",
    "Reduction potential of potassium is",
    "Drinking can be induced by",
    "Endemic typhus is transmitted by",
    "In Familial hypercholesterolemia there is",
    "Corynebacterum other than diphtheriae carrying toxin:",
    "Which of the following insecticides is a natural Which of the product ?",
    "Radiofrequency ablation treatment is most useful in:",
    "Which of the following will cause Bull's eye retinopathy",
    "Which of the following is lined by transitional epithelium ?",
    "Reversible cause of dementia is ?",
    "Donepezil is used in treatment of which disease?",
    "Rivastigmine is given in which disease?",
    "Alzheimer's disease, which is involved?",
    "Senile plaques in brain is a feature of which disease?",
    "Defect in Amyloid protein folding occurs in which disease?",
    "Regions of 'trinucleotide repeats' are seen in which disease?",
    "Alzheimer's Disease is associated with :",
    "Tau proteins are most commonly associated with ",
    " MMSE is used for the diagnosis of :",
    "The nucleus involved in Alzheimer's disease is -",
    "In Alzheimer's disease, plaque is made up of:",
    "Hirano bodies seen in?",
    "Which drug is not used now in Alzheimer's disease?",
    "Misfolded amyloid deposition in brain is seen in?",
    "Anticholinesterases drugs are used in which disease?",
    "'Silent epidemic' of the century is:",
    "Galantamine is used in ?"
    ]

In [4]:
generated_examples = generate_examples(prompts)

# Save generated examples to import in Label Studio
with open('ls_input_data.json', 'w') as f:
    json.dump(generated_examples, f, indent=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{
  "prompt": "Among which hypoxia AV 0 difference is max ?",
  "answer1": "for the hypoxiologic hypothesis, and a mean difference in hypoxial concentrations between subjects with no hypoxia (18 \u03bcg/dL and 0.3 ppm) and those with significant",
  "answer2": "This means AV 0 or no. On the other hand, if AV 0 is greater than no AV, then no AV is required to treat AV 0 or AV 0 difference.\n\nExceptions"
}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{
  "prompt": "Which of the following needs cholesterol and other lipids for growth?",
  "answer1": "Do they help the liver or muscles of the heart be more hydrated? Have these changes caused atherosclerosis. They are usually not.\n\nCholesterol or lipids may be",
  "answer2": "Are you having trouble, or just feel like it doesn't add up?\n\nYou know that one of the most commonly asked questions about all types of cholesterol is \"which one has"
}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

{
  "prompt": "Misfolded amyloid deposition in brain is seen in?",
  "answer1": ".\n\nIn vitro experiments are possible to confirm that amyloid deposition in humans is caused by a genetic defect. If this defect is present, its cause might be an inherited condition",
  "answer2": "I think that's what this is related to.\"\n\n\"It's not,\" said Dr. Mihail. \"It's the same thing they're doing with some people with"
}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{
  "prompt": "Anticholinesterases drugs are used in which disease?",
  "answer1": "is an isolated pathogen for which therapies can be developed. There have been studies with patients with metastatic kidney disease [21,23], which were shown to result in increased protein synthesis",
  "answer2": "Do they have direct physical impact on liver function?\n\nWhat if they can be taken after cancer is treated with chemotherapy drugs like tetracycline?\n\nIt"
}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{
  "prompt": "'Silent epidemic' of the century is:",
  "answer1": "the lack of awareness of the problems affecting the disabled and non-disabled community. The world is not looking for solutions to poverty, unemployment, poverty rates, illness and even death. It is not looking",
  "answer2": "\"It was just a coincidence that the first major mass killer had come just before the onset of the Cold War. It was not surprising that he was the first to come here. One of the big"
}
{
  "prompt": "Galantamine is used in ?",
  "answer1": "-methamphetamine?, methamphetamine use, and cocaine use. It also has been shown that methylphenidate is particularly toxic to people who have previously been treated. The only other known instance of methylphenidate was",
  "answer2": "/m? as a stimulant and non-psychotropic. It is used in people who smoke cigarettes, so it is important in their daily dose. Because dopamine levels increase over time due to smoking, caffeine or"
}


In [6]:
with open('ls_input_data.json', 'w') as f:
    json.dump(generated_examples, f, indent=2)

In [7]:
import json

try:
    with open('ls_input_data.json', 'w') as f:
        json.dump(generated_examples, f, indent=2)
    print("File 'ls_input_data.json' created and data written successfully.")
except Exception as e:
    print("An error occurred:", str(e))

File 'ls_input_data.json' created and data written successfully.


In [8]:
import os
import json

# Print the current working directory (path)
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

try:
    with open('ls_input_data.json', 'w') as f:
        json.dump(generated_examples, f, indent=2)
    print("File 'ls_input_data.json' created and data written successfully.")
except Exception as e:
    print("An error occurred:", str(e))


Current Working Directory: /content/trlx/examples/summarize_rlhf/reward_model
File 'ls_input_data.json' created and data written successfully.
